# 9.30 Gradient and Newton methods

minimize
f (x) = −
P m
i=1
log(1 − a Ti x) −
P n
i=1
log(1 − x 2 i ),

a) Use the gradient method to solve the problem, using reasonable choices for the back-
tracking parameters, and a stopping criterion of the form k∇f (x)k 2 ≤ η. Plot the
objective function and step length versus iteration number. (Once you have deter-
mined p ⋆ to high accuracy, you can also plot f − p ⋆ versus iteration.) Experiment
with the backtracking parameters α and β to see their effect on the total number of
iterations required. Carry these experiments out for several instances of the problem,
of different sizes.

(b) Repeat using Newton’s method, with stopping criterion based on the Newton decre-
ment λ 2 . Look for quadratic convergence. You do not have to use an efficient method
to compute the Newton step, as in exercise 9.27; you can use a general purpose dense
solver, although it is better to use one that is based on a Cholesky factorization.

In [ ]:
def gradient_descent(f, df, alpha, beta):
    eta=1e-5 # Stopping criterion
    
    while df(x) < eta:
        # 1. Determine a descent direction ∆x
        deltax = df(x)
        # 2. Line search: choose a step size t > 0
        while f(x + t*deltax) > f(x) + alpha*t*df(x)*deltax:
                t = beta*t
        # 3. Update: x := x + t∆x
        x = x + t*df(x)
        
    return x, f(x)
    

In [ ]:
def newton_method(f, df, ddf, alpha, beta):
    epsilon = 1e-5
    
    lambd = 1e10
    while lambd/2 < epsilon:
        # 1. Determine a descent direction ∆x
        deltax := −inverse(ddf(x))*df(x)
        lambd = -df(x)*deltax
        # 2. Line search: choose a step size t > 0
        while f(x + t*deltax) > f(x) + alpha*t*df(x)*deltax:
                t = beta*t
                
        # 3. Update: x := x + t∆x
        x = x + t*df(x)